In [832]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [3]:
labels = pd.read_csv("fiuba-trocafone-tp2-final-set/labels_training_set.csv")
df = pd.read_csv("fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
df_persons = pd.read_csv("tp2-github/data/trocafone_kaggle_test.csv")

In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day

In [5]:
df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'month', 'day'],
      dtype='object')

In [6]:
df["event"].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

### Dia, mes con mayor cantidad de visitas de cada persona

In [7]:
# Dia con mas entradas y su cantidad de entradas

entradas_x_dia = df.groupby("person")['day'].value_counts().rename("cant_dia_freq").reset_index()
entradas_x_dia.columns = ['person', 'day', 'cant_dia_freq']
entradas_x_dia = entradas_x_dia.sort_values('cant_dia_freq',ascending=False)
dia_mas_entradas = entradas_x_dia.drop_duplicates(subset=['person'])
dia_mas_entradas.head()

,person,day,cant_dia_freq
99325,c76b8417,3,652
28266,37eff05b,29,616
112559,e1443dd4,30,572
51164,656c18ef,10,512
41082,50e16a8a,19,423


In [8]:
# Mes con mas entradas y su cantidad de entradas

entradas_x_mes = df.groupby("person")['month'].value_counts().rename("cant_mes").reset_index()
entradas_x_mes.columns = ['person', 'month', 'cant_mes']
entradas_x_mes = entradas_x_mes.sort_values('cant_mes',ascending=False)
mes_mas_entradas = entradas_x_mes.drop_duplicates(subset=['person'])
mes_mas_entradas.head()

,person,month,cant_mes
38966,c76b8417,5,3028
18217,5c76e694,5,1609
32890,a7ffa917,5,1604
4824,18489dd5,5,1563
9989,33385551,4,1539


In [9]:
# Merge

cant_mes_mas_entradas = mes_mas_entradas[["person", "cant_mes"]]
cant_dia_mas_entradas = dia_mas_entradas[["person", "cant_dia_freq"]]

mes_dia_mas_entradas = pd.merge(cant_mes_mas_entradas, cant_dia_mas_entradas, on="person", how="inner")

df_con_labels = pd.merge(mes_dia_mas_entradas, labels, on="person", how="inner")
df_con_labels.head()

,person,cant_mes,cant_dia_freq,label
0,5c76e694,1609,191,0
1,18489dd5,1563,174,0
2,1775ba85,1450,126,0
3,6abd2bf1,1334,300,0
4,97b0c0d1,1328,217,0


In [144]:
mes_dia_mas_entradas_predecir = pd.merge(mes_dia_mas_entradas, df_persons, on="person", how="inner")

In [145]:
mes_dia_mas_entradas_predecir["person"].count() - df_persons["person"].count()

0

In [10]:
df_con_labels_num = df_con_labels[["cant_mes", "cant_dia_freq", "label"]]

## Prueba

In [11]:
# Separamos la variable a predecir

X, y = df_con_labels_num.iloc[:,:-1],df_con_labels_num.iloc[:,-1]

In [12]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [13]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [14]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [15]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [16]:
# Predecimos

preds = xg_reg.predict(X_test)

In [17]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9303328826218531
Test acuracy:  0.927890806077775
ROC auc score:  0.5285741205381574
Confusion matrix: 
[[3586   86]
 [ 194   17]]


### Entradas en los ultimos 30 dias (hasta la ultima visita)

In [18]:
df_meses = df[["timestamp", "person", "month"]]
df_ultimo_dia = df_meses.groupby("person")["timestamp"].max().reset_index()
df_ultimo_dia.columns = ["person", "last_day"]
df_meses_ultimo_dia = pd.merge(df_ultimo_dia, df_meses, on="person", how="inner")
entradas_30_dias = df_meses_ultimo_dia.loc[((df_meses_ultimo_dia["last_day"] - timedelta(days=30)) < \
                                           df_meses_ultimo_dia["timestamp"]) & (df_meses_ultimo_dia["timestamp"]< \
                                           df_meses_ultimo_dia["last_day"])]
entradas_30_dias.head()

,person,last_day,timestamp,month
1,0008ed71,2018-05-17 16:28:37,2018-05-17 12:27:47,5
2,0008ed71,2018-05-17 16:28:37,2018-05-17 13:45:00,5
3,0008ed71,2018-05-17 16:28:37,2018-05-17 16:22:06,5
4,0008ed71,2018-05-17 16:28:37,2018-05-17 13:44:59,5
5,0008ed71,2018-05-17 16:28:37,2018-05-17 16:21:54,5


In [19]:
entradas_30_dias_df = entradas_30_dias["person"].value_counts().rename("entradas_30_dias").reset_index()
entradas_30_dias_df.columns = ["person", "entradas_30_dias"]
entradas_30_dias_df.head()

,person,entradas_30_dias
0,c76b8417,3027
1,5c76e694,1608
2,a7ffa917,1603
3,18489dd5,1562
4,622b4acf,1458


In [20]:
# Merge

df_con_labels2 = pd.merge(entradas_30_dias_df, df_con_labels, on="person", how="inner")
df_con_labels2.head()

,person,entradas_30_dias,cant_mes,cant_dia_freq,label
0,5c76e694,1608,1609,191,0
1,18489dd5,1562,1563,174,0
2,6abd2bf1,1319,1334,300,0
3,97b0c0d1,1306,1328,217,0
4,595b9b50,1177,1179,377,0


In [146]:
entradas_30_dias_predecir = pd.merge(entradas_30_dias_df, df_persons, on="person", how="inner")
entradas_30_dias_predecir["person"].count() - df_persons["person"].count()

-442

In [21]:
df_con_labels2_num = df_con_labels2[["entradas_30_dias", "cant_mes", "cant_dia_freq", "label"]]

## Prueba

In [22]:
# Separamos la variable a predecir

X, y = df_con_labels2_num.iloc[:,:-1], df_con_labels2_num.iloc[:,-1]

In [23]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [24]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [25]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [26]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
# Predecimos

preds = xg_reg.predict(X_test)

In [28]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9202603207993689
Test acuracy:  0.9103339468840389
ROC auc score:  0.521793575484172
Confusion matrix: 
[[3444  144]
 [ 197   18]]


## Entradas en el ultimo mes (mes 5)

In [167]:
todas_las_personas = df_meses[["person"]].drop_duplicates(subset="person")
entradas_mes_5 = df_meses.loc[df_meses["month"] == 5]
cant_entradas_mes_5 = entradas_mes_5.groupby("person").agg({"month": "count"}).reset_index()
cant_entradas_mes_5.columns = ["person", "cant_mes_5"]
cant_entradas_mes_5 = pd.merge(cant_entradas_mes_5, todas_las_personas, on="person", how="right").fillna(0)
cant_entradas_mes_5.head()

,person,cant_mes_5
0,0008ed71,6.0
1,00091926,448.0
2,000ba417,206.0
3,000c79fe,17.0
4,000e4d9e,411.0


In [162]:
# Merge

df_con_mes_5 = pd.merge(cant_entradas_mes_5, df_con_labels2, on="person", how="inner")
df_con_mes_5.head()

,person,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6.0,5,6,6,0
1,000c79fe,17.0,16,17,17,0
2,001802e4,19.0,18,19,19,0
3,0019e639,290.0,358,290,72,0
4,001b0bf9,7.0,6,7,7,0


In [163]:
# Veo cuantas personas a predecir pierdo
entradas_mes_5_predecir = pd.merge(cant_entradas_mes_5, df_persons, on="person", how="inner")
entradas_mes_5_predecir["person"].count() - df_persons["person"].count()

0

## Prueba

In [915]:
# Separamos la variable a predecir
df_con_mes_5_num = df_con_mes_5.drop(columns="person")

X, y = df_con_mes_5_num.iloc[:,:-1],df_con_mes_5_num.iloc[:,-1]

In [916]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [917]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [918]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [919]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [920]:
# Predecimos

preds = xg_reg.predict(X_test)

In [921]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8679332106231922
Test acuracy:  0.8645805942676834
ROC auc score:  0.5890356751491538
Confusion matrix: 
[[3234  378]
 [ 137   54]]


## Cantidad de entradas totales

In [31]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)
entradas_tot.head()

,person,cant_entradas
0,0008ed71,6
1,00091926,448
2,00091a7a,10
3,000ba417,206
4,000c79fe,17


In [32]:
# Merge

df_con_entradas = pd.merge(entradas_tot, df_con_mes_5, on="person", how="inner")
df_con_entradas.head()

,person,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6,6,5,6,6,0
1,000c79fe,17,17,16,17,17,0
2,001802e4,19,19,18,19,19,0
3,0019e639,471,290,358,290,72,0
4,001b0bf9,7,7,6,7,7,0


In [164]:
# Veo cuantas personas a predecir pierdo
entradas_tot_predecir = pd.merge(entradas_tot, df_persons, on="person", how="inner")
entradas_tot_predecir["person"].count() - df_persons["person"].count()

0

## Prueba

In [922]:
# Separamos la variable a predecir
df_con_entradas_num = df_con_entradas.drop(columns="person")

X, y = df_con_entradas_num.iloc[:,:-1],df_con_entradas_num.iloc[:,-1]

In [923]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [924]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [925]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [926]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [927]:
# Predecimos

preds = xg_reg.predict(X_test)

In [928]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9041838134430727
Test acuracy:  0.8973936899862826
ROC auc score:  0.5427985467769387
Confusion matrix: 
[[3246  236]
 [ 138   25]]


### Cantidad de checkouts

In [168]:
checkouts = df.loc[df["event"] == "checkout"]

checkouts_tot = checkouts.groupby("person")["event"].value_counts().rename("check_tot").reset_index()
cant_checkouts_tot = checkouts_tot[["person", "check_tot"]]
cant_checkouts_tot = pd.merge(cant_entradas_mes_5, todas_las_personas, on="person", how="right").fillna(0)

In [169]:
# Merge

df_con_check = pd.merge(cant_checkouts_tot, df_con_entradas, on="person", how="inner")
df_con_check.head()

,person,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6.0,6,6,5,6,6,0
1,000c79fe,17.0,17,17,16,17,17,0
2,001802e4,19.0,19,19,18,19,19,0
3,0019e639,290.0,471,290,358,290,72,0
4,001b0bf9,7.0,7,7,6,7,7,0


In [170]:
# xgboost solo usa valores numericos
df_con_check_num = df_con_check.drop(columns="person")

In [171]:
# Veo cuantas personas a predecir pierdo
cant_checkouts_tot_predecir = pd.merge(cant_checkouts_tot, df_persons, on="person", how="inner")
cant_checkouts_tot_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [36]:
# Separamos la variable a predecir

X, y = df_con_check_num.iloc[:,:-1], df_con_check_num.iloc[:,-1]

In [37]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [38]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [39]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [40]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [41]:
# Predecimos

preds = xg_reg.predict(X_test)

In [42]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9448868071818891
Test acuracy:  0.9375585388698096
ROC auc score:  0.5753296607423287
Confusion matrix: 
[[2988  138]
 [  62   15]]


## Cantidad de entradas al evento mas visitado por cada persona

In [43]:
s = df.groupby("person")['event'].value_counts().rename().reset_index()
s.columns = ['person', 'event', 'cant_ev']
s = s.sort_values('cant_ev',ascending=False)
mayor_evento = s.drop_duplicates(subset=['person']).drop(columns="event")
mayor_evento.head()

,person,cant_ev
97161,6abd2bf1,2355
160896,b1f4dbf6,2233
129961,8fb4929e,1912
141831,9ccf882a,1891
6202,06ed04d6,1881


In [176]:
# Cantidad de entradas al modelo mas visitado por cada persona

mod = df.groupby("person")['model'].value_counts().rename().reset_index()
mod.columns = ['person', 'model', 'cant_mod']
mod = mod.sort_values('cant_mod',ascending=False)
mayor_modelo = mod.drop_duplicates(subset=['person']).drop(columns="model")
mayor_modelo = pd.merge(mayor_modelo, todas_las_personas, on="person", how="right").fillna(0)
mayor_modelo.head()

,person,cant_mod
0,b1f4dbf6,836.0
1,171e75cb,683.0
2,6f19cfd9,624.0
3,6abd2bf1,607.0
4,9bf968c5,568.0


In [177]:
# Cantidad de entradas al search engine mas visitado por cada persona

eng = df.groupby("person")['search_engine'].value_counts().rename().reset_index()
eng.columns = ['person', 'search_engine', 'cant_eng']
eng = eng.sort_values('cant_eng',ascending=False)
mayor_engine = eng.drop_duplicates(subset=['person']).drop(columns="search_engine")
mayor_engine = pd.merge(mayor_engine, todas_las_personas, on="person", how="right").fillna(0)
mayor_engine.head()

,person,cant_eng
0,c76b8417,762.0
1,6ca3126e,245.0
2,622b4acf,206.0
3,25b77cf2,161.0
4,5c76e694,136.0


In [178]:
# Cantidad de entradas al campaign source mas visitado por cada persona

src = df.groupby("person")['campaign_source'].value_counts().rename().reset_index()
src.columns = ['person', 'campaign_source', 'cant_src']
src = src.sort_values('cant_src',ascending=False)
mayor_camp = src.drop_duplicates(subset=['person']).drop(columns="campaign_source")
mayor_camp = pd.merge(mayor_camp, todas_las_personas, on="person", how="right").fillna(0)
mayor_camp.head()

,person,cant_src
0,02f14240,500.0
1,c76b8417,374.0
2,6ca3126e,335.0
3,622b4acf,282.0
4,3b2d17f6,221.0


In [179]:
# Cantidad de entradas desde la ciudad mas frecuente de cada persona

cty = df.groupby("person")['city'].value_counts().rename().reset_index()
cty.columns = ['person', 'city', 'cant_city']
src = cty.sort_values('cant_city',ascending=False)
mayor_ciudad = src.drop_duplicates(subset=['person']).drop(columns="city")
mayor_ciudad = pd.merge(mayor_ciudad, todas_las_personas, on="person", how="right").fillna(0)
mayor_ciudad.head()

,person,cant_city
0,5059f7fd,268.0
1,7ac0c607,208.0
2,67bdc946,207.0
3,ffee0f18,173.0
4,9b3b43aa,171.0


In [180]:
# Merge

from functools import reduce

# merge de todos los dfs

dfs = [mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento]
df_final1 = reduce(lambda left,right: pd.merge(left,right,on='person', how='inner'), dfs)
df_final = pd.merge(df_final1, df_con_check, on="person", how="inner")
df_final.head()

,person,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,ffee0f18,173.0,76.0,65.0,80.0,1518,218.0,3458,218,217,1028,282,0
1,9b3b43aa,171.0,17.0,4.0,468.0,1289,246.0,1663,246,234,923,174,0
2,02f14240,155.0,500.0,6.0,272.0,523,225.0,1798,225,282,847,114,0
3,345fc18e,153.0,155.0,10.0,106.0,844,367.0,1569,367,364,1007,137,0
4,30ec9daf,152.0,30.0,10.0,83.0,423,185.0,803,185,184,189,68,0


In [182]:
df_final_num = df_final.drop(columns="person")

In [181]:
# Veo cuantas personas a predecir pierdo
df1_predecir = pd.merge(df_final1, df_persons, on="person", how="inner")
df1_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [183]:
# Separamos la variable a predecir

X, y = df_final_num.iloc[:,:-1], df_final_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [184]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [185]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [186]:
# Predecimos

preds = xg_reg.predict(X_test)

In [187]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8734567901234568
Test acuracy:  0.8716049382716049
ROC auc score:  0.596550004757156
Confusion matrix: 
[[3129  353]
 [ 115   48]]


## Distancia entre primer y ultimo dia de entrada al sitio

In [55]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [56]:
# Merge

df_con_distancias = pd.merge(distancia_dias, df_final, on="person", how="inner") 
df_con_distancias_num = df_con_distancias.drop(columns="person")
df_con_distancias_num.head()

,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0,1,1,1,4,9,1,17,17,16,17,17,0
1,114,12,17,12,63,189,15,471,290,358,290,72,0
2,79,8,4,6,20,52,1,96,54,64,54,17,0
3,67,5,10,10,8,17,3,54,5,26,46,22,0
4,62,2,2,1,3,17,1,38,26,25,26,26,0


In [188]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [189]:
# Separamos la variable a predecir

X, y = df_con_distancias_num.iloc[:,:-1], df_con_distancias_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [190]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [191]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [192]:
# Predecimos

preds = xg_reg.predict(X_test)

In [193]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9115798295851376
Test acuracy:  0.9067039106145252
ROC auc score:  0.59551431955538
Confusion matrix: 
[[1608  125]
 [  42   15]]


## Cantidad de modelos distintos vistos por cada usuario

In [62]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [63]:
# Merge 

df_con_modelos = pd.merge(cant_modelos, df_con_distancias, on="person", how="inner")
df_con_modelos_num = df_con_modelos.drop(columns="person")
df_con_modelos_num.head()

,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,1,0,1,1,1,4,9,1,17,17,16,17,17,0
1,26,114,12,17,12,63,189,15,471,290,358,290,72,0
2,6,79,8,4,6,20,52,1,96,54,64,54,17,0
3,6,67,5,10,10,8,17,3,54,5,26,46,22,0
4,9,62,2,2,1,3,17,1,38,26,25,26,26,0


In [194]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [64]:
# Separamos la variable a predecir

X, y = df_con_modelos_num.iloc[:,:-1], df_con_modelos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [66]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
# Predecimos

preds = xg_reg.predict(X_test)

In [68]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8923033943288169
Test acuracy:  0.8759776536312849
ROC auc score:  0.5881292961196991
Confusion matrix: 
[[1552  181]
 [  41   16]]


## Cantidad de días distintos de entrada al sitio

In [69]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [70]:
# Merge 

df_con_dias_dist = pd.merge(cant_dias_dist, df_con_modelos, on="person", how="inner")
df_con_dias_dist_num = df_con_dias_dist.drop(columns=["person", "modelos_dist"])
df_con_dias_dist_num.head()

,cant_dias_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,1,0,1,1,1,4,9,1,17,17,16,17,17,0
1,14,114,12,17,12,63,189,15,471,290,358,290,72,0
2,11,79,8,4,6,20,52,1,96,54,64,54,17,0
3,6,67,5,10,10,8,17,3,54,5,26,46,22,0
4,2,62,2,2,1,3,17,1,38,26,25,26,26,0


In [195]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [71]:
# Separamos la variable a predecir

X, y = df_con_dias_dist_num.iloc[:,:-1], df_con_dias_dist_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [72]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [73]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [74]:
# Predecimos

preds = xg_reg.predict(X_test)

In [75]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8868557061042045
Test acuracy:  0.8698324022346369
ROC auc score:  0.6104058472783227
Confusion matrix: 
[[1538  195]
 [  38   19]]


## Promedio de entradas por dia

In [76]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)

df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()

entradas_tot_y_dias = pd.merge(entradas_tot, cant_dias_dist, on="person", how="inner")
entradas_tot_y_dias["promedio_por_dia"] = (entradas_tot_y_dias["cant_dias_dist"] / entradas_tot_y_dias["cant_entradas"])
entradas_tot_y_dias.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,person,cant_entradas,cant_dias_dist,promedio_por_dia
0,0008ed71,6,1,0.166667
1,00091926,448,22,0.049107
2,00091a7a,10,1,0.100000
3,000ba417,206,3,0.014563
4,000c79fe,17,1,0.058824


In [77]:
entradas_tot_y_dias = entradas_tot_y_dias[["person", "promedio_por_dia"]]

In [196]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_dias_predecir = pd.merge(entradas_tot_y_dias, df_persons, on="person", how="inner")
entradas_tot_y_dias_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por mes

In [952]:
df_mes = df[["person", "month"]]
cant_meses_dist = df_mes.groupby("person")["month"].nunique().rename("cant_meses_dist").reset_index()

entradas_tot_y_meses = pd.merge(entradas_tot, cant_meses_dist, on="person", how="inner")
entradas_tot_y_meses["promedio_por_mes"] = (entradas_tot_y_meses["cant_meses_dist"] / entradas_tot_y_meses["cant_entradas"])
entradas_tot_y_meses = entradas_tot_y_meses[["person", "promedio_por_mes"]]

entradas_tot_y_meses.head()

,person,promedio_por_mes
0,0008ed71,0.166667
1,00091926,0.002232
2,00091a7a,0.100000
3,000ba417,0.004854
4,000c79fe,0.058824


In [953]:
# Merge

labels_con_promedios = pd.merge(entradas_tot_y_meses, entradas_tot_y_dias, on="person", how="inner")
labels_con_promedios = pd.merge(labels_con_promedios, df_con_dias_dist, on="person", how="inner")
labels_con_promedios_num = labels_con_promedios.drop(columns="person")
labels_con_promedios_num.head()

,promedio_por_mes,promedio_por_dia,cant_dias_dist,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0.058824,0.058824,1,1,0,1,1,1,4,9,1,17,17,16,17,17,0
1,0.008493,0.029724,14,26,114,12,17,12,63,189,15,471,290,358,290,72,0
2,0.031250,0.114583,11,6,79,8,4,6,20,52,1,96,54,64,54,17,0
3,0.055556,0.111111,6,6,67,5,10,10,8,17,3,54,5,26,46,22,0
4,0.052632,0.052632,2,9,62,2,2,1,3,17,1,38,26,25,26,26,0


In [954]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_meses_predecir = pd.merge(entradas_tot_y_meses, df_persons, on="person", how="inner")
entradas_tot_y_meses_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [1036]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1037]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=5)


In [1038]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1039]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1040]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8833635982679146
Test acuracy:  0.8804469273743016
ROC auc score:  0.6328544963100191
Confusion matrix: 
[[1555  178]
 [  36   21]]


In [1041]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8833635982679146
Test acuracy:  0.8804469273743016

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1733
           1       0.11      0.37      0.16        57

   micro avg       0.88      0.88      0.88      1790
   macro avg       0.54      0.63      0.55      1790
weighted avg       0.95      0.88      0.91      1790


Precision Score:  0.10552763819095477
Recall Score:  0.3684210526315789
F1 Score:  0.1640625
Cohen Kappa Score:  0.12052236921254311
ROC auc score:  0.6328544963100191
Confusion matrix: 
[[1555  178]
 [  36   21]]


### Duplico los compradores (label = 1)

### Prueba

In [1172]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1173]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1174]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [1175]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1176]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1177]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.681266846361186
Test acuracy:  0.6849162011173184
ROC auc score:  0.6676081432664177
Confusion matrix: 
[[1189  544]
 [  20   37]]


In [1178]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.681266846361186
Test acuracy:  0.6849162011173184

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.69      0.81      1733
           1       0.06      0.65      0.12        57

   micro avg       0.68      0.68      0.68      1790
   macro avg       0.52      0.67      0.46      1790
weighted avg       0.95      0.68      0.79      1790


Precision Score:  0.06368330464716007
Recall Score:  0.6491228070175439
F1 Score:  0.11598746081504703
Cohen Kappa Score:  0.06156057059675446
ROC auc score:  0.6676081432664177
Confusion matrix: 
[[1189  544]
 [  20   37]]


### Usando SMOTE

In [1137]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1145]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=90, sampling_strategy='minority')
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [1146]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [1147]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1148]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1149]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.6190924905769788
Test acuracy:  0.25977653631284914
ROC auc score:  0.6007481195776515
Confusion matrix: 
[[ 410 1323]
 [   2   55]]


In [1150]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6190924905769788
Test acuracy:  0.25977653631284914

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.24      0.38      1733
           1       0.04      0.96      0.08        57

   micro avg       0.26      0.26      0.26      1790
   macro avg       0.52      0.60      0.23      1790
weighted avg       0.96      0.26      0.37      1790


Precision Score:  0.03991291727140784
Recall Score:  0.9649122807017544
F1 Score:  0.07665505226480837
Cohen Kappa Score:  0.016507170882889777
ROC auc score:  0.6007481195776515
Confusion matrix: 
[[ 410 1323]
 [   2   55]]


### Random oversampling

In [1151]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1159]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2)
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [1160]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [1161]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1162]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1163]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.5532962576758197
Test acuracy:  0.13072625698324022
ROC auc score:  0.5510675129832661
Confusion matrix: 
[[ 177 1556]
 [   0   57]]


In [1158]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.5074730622175878
Test acuracy:  0.04860335195530726

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.02      0.03      1733
           1       0.03      1.00      0.06        57

   micro avg       0.05      0.05      0.05      1790
   macro avg       0.52      0.51      0.05      1790
weighted avg       0.97      0.05      0.03      1790


Precision Score:  0.03238636363636364
Recall Score:  1.0
F1 Score:  0.06274078150798018
Cohen Kappa Score:  0.0011206537802405103
ROC auc score:  0.5086555106751298
Confusion matrix: 
[[  30 1703]
 [   0   57]]


### Random undersampling

In [1164]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1166]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler(random_state=2)
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [1167]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [1168]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1169]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1170]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.5424528301886793
Test acuracy:  0.08659217877094973
ROC auc score:  0.5282746682054241
Confusion matrix: 
[[  98 1635]
 [   0   57]]


In [1171]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.5424528301886793
Test acuracy:  0.08659217877094973

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.06      0.11      1733
           1       0.03      1.00      0.07        57

   micro avg       0.09      0.09      0.09      1790
   macro avg       0.52      0.53      0.09      1790
weighted avg       0.97      0.09      0.11      1790


Precision Score:  0.03368794326241135
Recall Score:  1.0
F1 Score:  0.06518010291595197
Cohen Kappa Score:  0.0038028171890603923
ROC auc score:  0.5282746682054241
Confusion matrix: 
[[  98 1635]
 [   0   57]]


## Cantidad de eventos particulares

In [85]:
df_events = df[["person", "event"]]
cant_por_evento = pd.concat([pd.get_dummies(df_events['event']),df_events[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()
cant_por_evento_short = cant_por_evento[["person", "ad campaign hit", "search engine hit"]]

# Merge ... "search engine hit", "generic listing", "searched products" "ad campaign hit", "brand listing"

labels_con_eventos = pd.merge(cant_por_evento_short, labels_con_promedios, on="person", how="inner")
labels_con_eventos_num = labels_con_eventos.drop(columns="person")
labels_con_eventos_num.head()

,ad campaign hit,search engine hit,promedio_por_mes,promedio_por_dia,cant_dias_dist,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,1.0,1.0,0.058824,0.058824,1,1,0,1,1,1,4,9,1,17,17,16,17,17,0
1,29.0,13.0,0.008493,0.029724,14,26,114,12,17,12,63,189,15,471,290,358,290,72,0
2,7.0,6.0,0.031250,0.114583,11,6,79,8,4,6,20,52,1,96,54,64,54,17,0
3,13.0,10.0,0.055556,0.111111,6,6,67,5,10,10,8,17,3,54,5,26,46,22,0
4,2.0,1.0,0.052632,0.052632,2,9,62,2,2,1,3,17,1,38,26,25,26,26,0


In [198]:
# Veo cuantas personas a predecir pierdo
cant_por_evento_short_predecir = pd.merge(cant_por_evento_short, df_persons, on="person", how="inner")
cant_por_evento_short_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [543]:
# Separamos la variable a predecir

X, y = labels_con_eventos_num.iloc[:,:-1], labels_con_eventos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [587]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=15)


In [588]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=15, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [589]:
# Predecimos

preds = xg_reg.predict(X_test)

In [590]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8842017041486241
Test acuracy:  0.8776536312849162
ROC auc score:  0.6229284983954404
Confusion matrix: 
[[1551  182]
 [  37   20]]


## Visitas a productos segun condicion, color y almacenamiento de producto.¶
copiado de santi

In [201]:
df_events_visitas_prod = df.loc[df['event'] == 'viewed product']
df_events_visitas_prod = df_events_visitas_prod[['person','storage','color','condition']]

df_events_visitas_prod_condition_storage = pd.concat([df_events_visitas_prod['person'],\
           pd.get_dummies(df_events_visitas_prod['storage']),\
           pd.get_dummies(df_events_visitas_prod['condition'])],axis = 1).groupby('person').sum().reset_index()

In [202]:
top_10_colores = df_events_visitas_prod['color'].value_counts().head(10).index


df_events_visitas_prod_color = df_events_visitas_prod.loc[df_events_visitas_prod['color'].isin(top_10_colores)]
df_events_visitas_prod_color = pd.concat([df_events_visitas_prod_color['person'],\
                                          pd.get_dummies(df_events_visitas_prod_color['color'])],axis = 1)

df_events_visitas_prod_color = df_events_visitas_prod_color.groupby('person').sum().reset_index()

df_events_visitas_prod_color_storage_condition = pd.merge(df_events_visitas_prod_condition_storage,\
                        df_events_visitas_prod_color, on = 'person', how = 'inner')


df_events_visitas_prod_color_storage_condition = pd.merge(df_events_visitas_prod_color_storage_condition, todas_las_personas, on="person", how="right").fillna(0)
df_events_visitas_prod_color_storage_condition.head()

,person,128GB,16GB,256GB,32GB,4GB,512MB,64GB,8GB,Bom,...,Azul,Branco,Cinza espacial,Dourado,Ouro Rosa,Prata,Prateado,Preto,Preto Matte,Rosa
0,00091926,48.0,104.0,10.0,132.0,0.0,0.0,78.0,0.0,102.0,...,3.0,3.0,65.0,101.0,28.0,1.0,43.0,79.0,29.0,3.0
1,00091a7a,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,108.0,1.0,20.0,1.0,0.0,1.0,22.0,110.0,...,4.0,15.0,1.0,14.0,0.0,0.0,0.0,71.0,0.0,7.0
3,000c79fe,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,000e4d9e,1.0,108.0,1.0,208.0,0.0,0.0,21.0,0.0,124.0,...,23.0,85.0,14.0,46.0,2.0,6.0,1.0,149.0,0.0,9.0


In [203]:
# Veo cuantas personas a predecir pierdo
visitas_prod_color_storage_condition_predecir = pd.merge(df_events_visitas_prod_color_storage_condition, df_persons, on="person", how="inner")
visitas_prod_color_storage_condition_predecir["person"].count() - df_persons["person"].count()

0

In [204]:
# Merge

labels_con_storage_color = pd.merge(df_events_visitas_prod_color_storage_condition, labels_con_eventos, on="person", how="inner")
labels_con_storage_color_num = labels_con_storage_color.drop(columns="person")
labels_con_storage_color_num.head()

,128GB,16GB,256GB,32GB,4GB,512MB,64GB,8GB,Bom,Bom - Sem Touch ID,...,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1,4,9,1,17,17,16,17,17,0
1,0.0,94.0,0.0,14.0,0.0,0.0,0.0,81.0,105.0,0.0,...,12,63,189,15,471,290,358,290,72,0
2,8.0,19.0,4.0,11.0,0.0,0.0,10.0,0.0,9.0,0.0,...,6,20,52,1,96,54,64,54,17,0
3,1.0,8.0,0.0,4.0,0.0,0.0,3.0,1.0,9.0,5.0,...,10,8,17,3,54,5,26,46,22,0
4,0.0,5.0,0.0,2.0,0.0,0.0,0.0,5.0,12.0,0.0,...,1,3,17,1,38,26,25,26,26,0


### Prueba

In [631]:
# Separamos la variable a predecir

X, y = labels_con_storage_color_num.iloc[:,:-1], labels_con_storage_color_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [665]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=15)


In [666]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=15, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [667]:
# Predecimos

preds = xg_reg.predict(X_test)

In [668]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9234529962285235
Test acuracy:  0.9067039106145252
ROC auc score:  0.5351895357171299
Confusion matrix: 
[[1615  117]
 [  50    8]]


# Entro como 'new' en el mes 5

In [219]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["is_new"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "is_new"]]
df_mew_5 = df_mew_5.sort_values("is_new", ascending=True)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'])
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,is_new
0,4886f805,0.0
1,d6288188,0.0
2,4858040e,0.0
3,a6538d3e,0.0
4,b884c881,0.0


In [220]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

In [221]:
# Merge

labels_con_new_mes5 = pd.merge(df_mew_5, labels_con_storage_color, on="person", how="inner")
labels_con_new_mes5_num = labels_con_new_mes5.drop(columns="person")

### Prueba

In [669]:
# Separamos la variable a predecir

X, y = labels_con_new_mes5_num.iloc[:,:-1], labels_con_new_mes5_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [694]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=4)


In [695]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=4, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [696]:
# Predecimos

preds = xg_reg.predict(X_test)

In [697]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8963542394189132
Test acuracy:  0.8698324022346369
ROC auc score:  0.6033637873754153
Confusion matrix: 
[[1535  185]
 [  48   22]]


# -- Agrego cosas de Santi --

## Cantidad de visitas a los 15 modelos mas visitados

In [106]:
df_visitas_producto = df.loc[df['event'] == 'viewed product']
top_15 = df_visitas_producto['model'].value_counts().head(15).index
df_15_productos_mas_visitados = df_visitas_producto.loc[df_visitas_producto['model'].isin(top_15)]

df_15_productos_mas_visitados = df_15_productos_mas_visitados[['person','model']]
df_15_productos_mas_visitados.head()

,person,model
1,ad93850f,iPhone 5s
2,0297fc1e,iPhone 6S
3,2d681dd8,iPhone 7
6,1b9f7cf6,iPhone 6
7,29ebb414,iPhone 6 Plus


In [107]:
df_15_productos_mas_visitados = pd.concat([df_15_productos_mas_visitados,\
                    pd.get_dummies(df_15_productos_mas_visitados['model'])],axis = 1)

df_15_productos_mas_visitados = df_15_productos_mas_visitados.groupby('person').sum().reset_index()
df_15_productos_mas_visitados.head()

,person,Samsung Galaxy J5,Samsung Galaxy S6 Edge,Samsung Galaxy S6 Flat,Samsung Galaxy S7,Samsung Galaxy S7 Edge,Samsung Galaxy S8,iPhone 5c,iPhone 5s,iPhone 6,iPhone 6 Plus,iPhone 6S,iPhone 6S Plus,iPhone 7,iPhone 7 Plus,iPhone SE
0,00091926,1.0,3.0,15.0,1.0,9.0,5.0,0.0,0.0,5.0,41.0,94.0,51.0,45.0,9.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,000ba417,11.0,0.0,1.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,000e4d9e,1.0,5.0,139.0,22.0,2.0,9.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,11.0


In [229]:
# vis = visitas
nombres_columnas = ['vis_'+columna for columna in df_15_productos_mas_visitados.columns]
nombres_columnas[0] = 'person'
df_15_productos_mas_visitados.columns = nombres_columnas

df_15_productos_mas_visitados = pd.merge(df_15_productos_mas_visitados, todas_las_personas, on="person", how="right").fillna(0)

df_15_productos_mas_visitados.head()

,person,vis_vis_Samsung Galaxy J5,vis_vis_Samsung Galaxy S6 Edge,vis_vis_Samsung Galaxy S6 Flat,vis_vis_Samsung Galaxy S7,vis_vis_Samsung Galaxy S7 Edge,vis_vis_Samsung Galaxy S8,vis_vis_iPhone 5c,vis_vis_iPhone 5s,vis_vis_iPhone 6,vis_vis_iPhone 6 Plus,vis_vis_iPhone 6S,vis_vis_iPhone 6S Plus,vis_vis_iPhone 7,vis_vis_iPhone 7 Plus,vis_vis_iPhone SE
0,00091926,1.0,3.0,15.0,1.0,9.0,5.0,0.0,0.0,5.0,41.0,94.0,51.0,45.0,9.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,000ba417,11.0,0.0,1.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,000e4d9e,1.0,5.0,139.0,22.0,2.0,9.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,11.0


In [230]:
# Veo cuantas personas a predecir pierdo
df_15_productos_mas_visitados_predecir = pd.merge(df_15_productos_mas_visitados, df_persons, on="person", how="inner")
df_15_productos_mas_visitados_predecir["person"].count() - df_persons["person"].count()

0

In [231]:
df_train_sin_labels = pd.merge(df_15_productos_mas_visitados,cant_por_evento,on = 'person',how = 'outer')

df_train_sin_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 27 columns):
person                            38829 non-null object
vis_vis_Samsung Galaxy J5         38829 non-null float64
vis_vis_Samsung Galaxy S6 Edge    38829 non-null float64
vis_vis_Samsung Galaxy S6 Flat    38829 non-null float64
vis_vis_Samsung Galaxy S7         38829 non-null float64
vis_vis_Samsung Galaxy S7 Edge    38829 non-null float64
vis_vis_Samsung Galaxy S8         38829 non-null float64
vis_vis_iPhone 5c                 38829 non-null float64
vis_vis_iPhone 5s                 38829 non-null float64
vis_vis_iPhone 6                  38829 non-null float64
vis_vis_iPhone 6 Plus             38829 non-null float64
vis_vis_iPhone 6S                 38829 non-null float64
vis_vis_iPhone 6S Plus            38829 non-null float64
vis_vis_iPhone 7                  38829 non-null float64
vis_vis_iPhone 7 Plus             38829 non-null float64
vis_vis_iPhone SE                 388

In [232]:
# probar 2 alternativas
df_train_sin_labels1 = df_train_sin_labels.dropna()
df_train_sin_labels2 = df_train_sin_labels.fillna(0)

In [235]:
# Merge

labels_con_15_modelos = pd.merge(df_15_productos_mas_visitados, labels_con_new_mes5, on="person", how="inner")
labels_con_15_modelos_num = labels_con_15_modelos.drop(columns="person")

### Prueba

In [700]:
# Separamos la variable a predecir

X, y = labels_con_15_modelos_num.iloc[:,:-1], labels_con_15_modelos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [731]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=10)


In [732]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [733]:
# Predecimos

preds = xg_reg.predict(X_test)

In [734]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8914652884481072
Test acuracy:  0.8787709497206704
ROC auc score:  0.6136569380069524
Confusion matrix: 
[[1552  174]
 [  43   21]]


--- Aca bajó de 65 a 60!! ---

## Estado(region) del usuario

In [117]:
top_20_estados = list(df['region'].value_counts().head(21).index)
del top_20_estados[1]
top_20_estados

['Sao Paulo',
 'Minas Gerais',
 'Rio de Janeiro',
 'Bahia',
 'Pernambuco',
 'Ceara',
 'Parana',
 'Rio Grande do Sul',
 'Espirito Santo',
 'Federal District',
 'Maranhao',
 'Goias',
 'Santa Catarina',
 'Para',
 'Rio Grande do Norte',
 'Paraíba',
 'Piaui',
 'Alagoas',
 'Sergipe',
 'Amazonas']

In [118]:
df_estado = df[['person','region','month']]
df_estado = df_estado.sort_values('month',ascending = True).dropna()

df_estado = df_estado.loc[df_estado['region'].isin(top_20_estados)]


df_estado = df_estado.drop_duplicates(subset = 'person', keep = 'last').drop(columns = ['month'])

df_estado.head()

,person,region
2151588,0869b313,Pernambuco
2150498,78561ee9,Sao Paulo
2294065,540e67b1,Pernambuco
2248029,041627fe,Federal District
2151706,62acae45,Minas Gerais


In [119]:
# PROBAR 2 OPCIONES (LabelEncoder o con One hot encoding)
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_estado1 = df_estado.copy()

In [437]:
df_estado1['region'] = le.fit_transform(df_estado1['region'])
df_estado1 = pd.merge(df_estado1, todas_las_personas, on="person", how="right").fillna(0)

df_estado1.head()

,person,region
0,0869b313,12
1,78561ee9,18
2,540e67b1,12
3,041627fe,5
4,62acae45,8


In [429]:
df_estado2 = 0
df_estado2 = pd.concat([df_estado['person'],pd.get_dummies(df_estado['region'])],axis=1)
df_estado2 = pd.merge(df_estado2, todas_las_personas, on="person", how="right").fillna(0)
df_estado2.head()

,person,Alagoas,Amazonas,Bahia,Ceara,Espirito Santo,Federal District,Goias,Maranhao,Minas Gerais,...,Parana,Paraíba,Pernambuco,Piaui,Rio Grande do Norte,Rio Grande do Sul,Rio de Janeiro,Santa Catarina,Sao Paulo,Sergipe
0,0869b313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,78561ee9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,540e67b1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,041627fe,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,62acae45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [430]:
# Veo cuantas personas a predecir pierdo
df_estado2_predecir = pd.merge(df_estado2, df_persons, on="person", how="inner")
df_estado2_predecir["person"].count() - df_persons["person"].count()

0

In [438]:
# Veo cuantas personas a predecir pierdo
df_estado1_predecir = pd.merge(df_estado1, df_persons, on="person", how="inner")
df_estado1_predecir["person"].count() - df_persons["person"].count()

0

In [450]:
# Merge

labels_con_regiones = pd.merge(df_estado2, labels_con_15_modelos, on="person", how="inner")
labels_con_regiones_num = labels_con_regiones.drop(columns="person")

### Prueba

In [735]:
# Separamos la variable a predecir

X, y = labels_con_regiones_num.iloc[:,:-1], labels_con_regiones_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [740]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [741]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [742]:
# Predecimos

preds = xg_reg.predict(X_test)

In [743]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9460818550076826
Test acuracy:  0.9374301675977653
ROC auc score:  0.5083471760797342
Confusion matrix: 
[[1675   45]
 [  67    3]]


 --- ¿¿¿?? aca se cagó todo, paso a 0.5, wtf. Tanto con LE como con OHE ---

## Cantidad de veces que accede desde cada dispositivo

In [243]:
df_device = df.loc[df['event'] == 'visited site']

df_device = df_device[['person','device_type']]

df_device = pd.concat([df_device['person'],pd.get_dummies(df_device['device_type'])],axis = 1)

df_device.drop(columns = ['Unknown'],inplace = True)

df_device = df_device.groupby('person').sum().reset_index()

df_device = pd.merge(df_device, todas_las_personas, on="person", how="right").fillna(0)


df_device.head()

,person,Computer,Smartphone,Tablet
0,0008ed71,2.0,0.0,0.0
1,00091926,34.0,0.0,0.0
2,00091a7a,0.0,1.0,0.0
3,000ba417,6.0,0.0,0.0
4,000c79fe,0.0,1.0,0.0


In [129]:
# Lo mismo se podria hacer pero considerando las visitas a producto desde cada movil.
# Ver.

In [244]:
# Veo cuantas personas a predecir pierdo
df_device_predecir = pd.merge(df_device, df_persons, on="person", how="inner")
df_device_predecir["person"].count() - df_persons["person"].count()

0

In [245]:
# Merge

labels_con_dispositivos = pd.merge(df_device, labels_con_15_modelos, on="person", how="inner")
labels_con_dispositivos_num = labels_con_dispositivos.drop(columns="person")

### Prueba

In [744]:
# Separamos la variable a predecir

X, y = labels_con_dispositivos_num.iloc[:,:-1], labels_con_dispositivos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [777]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=20)


In [778]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=20, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [779]:
# Predecimos

preds = xg_reg.predict(X_test)

In [780]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8675792708478838
Test acuracy:  0.8664804469273742
ROC auc score:  0.634124983549468
Confusion matrix: 
[[1529  204]
 [  35   22]]


## Visitas a la pagina segun resolucion

In [283]:
df_resolucion = df.loc[df['event'] == 'visited site']
df_resolucion = df_resolucion[['person','screen_resolution']].dropna()

top_5_resoluciones = df_resolucion['screen_resolution'].value_counts().head(10).index
df_resolucion = df_resolucion.loc[df_resolucion['screen_resolution'].isin(top_5_resoluciones)]
df_resolucion = pd.concat([df_resolucion['person'],\
        pd.get_dummies(df_resolucion['screen_resolution'])],axis = 1).groupby('person').sum().reset_index()

df_resolucion = pd.merge(df_resolucion, todas_las_personas, on="person", how="right").fillna(0)

df_resolucion.head()

,person,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
0,0008ed71,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,00091926,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000ba417,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [270]:
df_resolucion.describe()

,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
count,32373.000000,32373.000000,32373.000000,32373.00000,32373.000000,32373.000000,32373.000000,32373.000000,32373.000000,32373.000000
mean,0.185031,0.204244,1.537083,0.17076,0.189664,0.245513,0.235752,0.136873,0.159763,2.262194
std,1.860610,2.087674,5.612604,1.75788,2.111903,2.313731,2.016357,1.443385,1.590214,7.019321
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,105.000000,133.000000,246.000000,99.00000,116.000000,146.000000,124.000000,78.000000,75.000000,293.000000


In [271]:
# Creo que es mucho 20 resoluciones, probar (quizas empeore)..

In [284]:
# Veo cuantas personas a predecir pierdo
df_resolucion_predecir = pd.merge(df_resolucion, df_persons, on="person", how="inner")
df_resolucion_predecir["person"].count() - df_persons["person"].count()

0

In [285]:
# Merge

labels_con_resoluciones = pd.merge(df_resolucion, labels_con_dispositivos, on="person", how="inner")
labels_con_resoluciones_num = labels_con_resoluciones.drop(columns="person")

In [274]:
labels_con_resoluciones.shape

(8949, 72)

In [141]:
# Super Merge
dfs=0
dfs =[mes_dia_mas_entradas, entradas_30_dias_df, df_con_mes_5, entradas_tot,\
      cant_checkouts_tot, mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento, df_con_check\
     , distancia_dias, cant_modelos, cant_dias_dist, entradas_tot_y_dias, entradas_tot_y_meses,\
      df_events_visitas_prod_color_storage_condition, df_mew_5, df_device, df_resolucion,\
      df_sistema]

df_mejor = pd.concat(dfs, axis=1).reset_index()

df_final = reduce(lambda left,right: pd.merge(left,right,on='person', how='inner'), dfs)
df_mejor.shape

NameError: name 'df_sistema' is not defined

### Prueba

In [781]:
# Separamos la variable a predecir

X, y = labels_con_resoluciones_num.iloc[:,:-1], labels_con_resoluciones_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [798]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=16)


In [799]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=16, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [800]:
# Predecimos

preds = xg_reg.predict(X_test)

In [801]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9187037295711692
Test acuracy:  0.9067039106145252
ROC auc score:  0.6144307481743921
Confusion matrix: 
[[1603  120]
 [  47   20]]


 --- Aumento un toke, bien ---

## Visitas a la pagina segun sistema operativo

In [977]:
df_sistema = df.loc[df['event'] == 'visited site']
df_sistema = df_sistema[['person','operating_system_version']].dropna()

top_5_sistemas = df_sistema['operating_system_version'].value_counts().head(5).index
df_sistema = df_sistema.loc[df_sistema['operating_system_version'].isin(top_5_sistemas)]
df_sistema = pd.concat([df_sistema['person'],\
        pd.get_dummies(df_sistema['operating_system_version'])],axis = 1).groupby('person').sum().reset_index()

df_sistema = pd.merge(df_sistema, todas_las_personas, on="person", how="right").fillna(0)
df_sistema.head()

,person,Android 6,Android 6.0.1,Android 7,Windows 10,Windows 7
0,0008ed71,0.0,0.0,0.0,2.0,0.0
1,00091926,0.0,0.0,0.0,0.0,34.0
2,000ba417,0.0,0.0,0.0,6.0,0.0
3,000c79fe,0.0,0.0,1.0,0.0,0.0
4,000e4d9e,0.0,0.0,0.0,13.0,0.0


In [978]:
# Veo cuantas personas a predecir pierdo
df_sistema_predecir = pd.merge(df_sistema, df_persons, on="person", how="inner")
df_sistema_predecir["person"].count() - df_persons["person"].count()

0

In [979]:
# Merge

labels_con_SO = pd.merge(df_sistema, labels_con_resoluciones, on="person", how="inner")
labels_con_SO_num = labels_con_SO.drop(columns="person")

In [980]:
labels_con_SO.shape

(8949, 77)

### Prueba

In [981]:
# Separamos la variable a predecir

X, y = labels_con_SO_num.iloc[:,:-1], labels_con_SO_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [982]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=9, scale_pos_weight=ratio)
ratio

25.514814814814816

In [983]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=9, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=25.514814814814816, seed=None, silent=True,
       subsample=1)

In [984]:
# Predecimos

preds = xg_reg.predict(X_test)

In [985]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8833635982679146
Test acuracy:  0.882122905027933
ROC auc score:  0.6152889399158056
Confusion matrix: 
[[1563  179]
 [  32   16]]


### Prueba

In [986]:
# Separamos la variable a predecir

X, y = labels_con_SO_num.iloc[:,:-1], labels_con_SO_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [987]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [988]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=10)
ratio

12.757407407407408

In [989]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [990]:
# Predecimos

preds = xg_reg.predict(X_test)

In [991]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.704401669134473
Test acuracy:  0.7
ROC auc score:  0.6635332950631458
Confusion matrix: 
[[1223  519]
 [  18   30]]


## Visitas a la pagina segun version del navegador

In [325]:
df_navegador = df.loc[df['event'] == 'visited site']
df_navegador = df_navegador[['person','browser_version']].dropna()

top_5_navegadores = df_navegador['browser_version'].value_counts().head(5).index
df_navegador = df_navegador.loc[df_navegador['browser_version'].isin(top_5_navegadores)]
df_navegador = pd.concat([df_navegador['person'],\
        pd.get_dummies(df_navegador['browser_version'])],axis = 1).groupby('person').sum().reset_index()

df_navegador = pd.merge(df_navegador, todas_las_personas, on="person", how="right").fillna(0)

df_navegador.head()

,person,Chrome 65.0,Chrome 66.0,Chrome Mobile 64.0,Chrome Mobile 65.0,Chrome Mobile 66.0
0,0008ed71,0.0,2.0,0.0,0.0,0.0
1,00091926,0.0,34.0,0.0,0.0,0.0
2,000ba417,0.0,6.0,0.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,1.0
4,000e619d,0.0,5.0,0.0,0.0,0.0


In [326]:
# Veo cuantas personas a predecir pierdo
df_navegador_predecir = pd.merge(df_navegador, df_persons, on="person", how="inner")
df_navegador_predecir["person"].count() - df_persons["person"].count()

0

In [327]:
# Merge

labels_con_navegador = pd.merge(df_navegador, labels_con_SO, on="person", how="inner")
labels_con_navegador_num = labels_con_navegador.drop(columns="person")

### Prueba

In [328]:
# Separamos la variable a predecir

X, y = labels_con_navegador_num.iloc[:,:-1], labels_con_navegador_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [329]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [330]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [331]:
# Predecimos

preds = xg_reg.predict(X_test)

In [332]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8446710434418215
Test acuracy:  0.8480446927374302
ROC auc score:  0.6010071663761378
Confusion matrix: 
[[1495  226]
 [  46   23]]


 --- Bajó, :( ---

## Visitas a producto por mes

In [339]:
df_visitas_producto_por_mes = df.loc[df_events['event'] == 'viewed product']
df_visitas_producto_por_mes = df_visitas_producto_por_mes[['person','month']]

df_visitas_producto_por_mes = pd.concat([pd.get_dummies(df_visitas_producto_por_mes['month'])\
                                         ,df_visitas_producto_por_mes[['person']]],axis = 1).groupby('person')\
                                        .sum().reset_index()

df_visitas_producto_por_mes = pd.merge(df_visitas_producto_por_mes, todas_las_personas, on="person", how="right").fillna(0)

df_visitas_producto_por_mes.head()


,person,1,2,3,4,5
0,00091926,0.0,0.0,0.0,0.0,372.0
1,00091a7a,0.0,0.0,3.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,153.0
3,000c79fe,0.0,0.0,0.0,0.0,3.0
4,000e4d9e,0.0,0.0,0.0,0.0,339.0


In [340]:
nombres_columnas = ['mes_'+str(columna) for columna in df_visitas_producto_por_mes.columns]
nombres_columnas[0] = 'person'
df_visitas_producto_por_mes.columns = nombres_columnas

# Merge 
labels_con_meses = pd.merge(df_visitas_producto_por_mes, labels_con_SO, on="person", how="inner")
labels_con_meses_num = labels_con_meses.drop(columns="person")

In [341]:
# Veo cuantas personas a predecir pierdo
df_visitas_producto_por_mes_predecir = pd.merge(df_visitas_producto_por_mes, df_persons, on="person", how="inner")
df_visitas_producto_por_mes_predecir["person"].count() - df_persons["person"].count()

0

In [ ]:
labels_con_meses.columns

### Prueba

In [342]:
# Separamos la variable a predecir

X, y = labels_con_meses_num.iloc[:,:-1], labels_con_meses_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [343]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [344]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [345]:
# Predecimos

preds = xg_reg.predict(X_test)

In [346]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8393630395306607
Test acuracy:  0.8312849162011173
ROC auc score:  0.5932299390344373
Confusion matrix: 
[[1469  265]
 [  37   19]]


## Dia de la semana que accede

In [998]:
df['day_of_week'] = df["timestamp"].dt.dayofweek
df_dia_visita = df.loc[df['event'] == 'visited site']
df_dia_visita = df_dia_visita[['person','day_of_week']]

df_dia_visita = pd.concat([df_dia_visita['person'],\
                                 pd.get_dummies(df_dia_visita['day_of_week'])],axis = 1)\
                                .groupby('person').sum().reset_index()

df_dia_visita = pd.merge(df_dia_visita, todas_las_personas, on="person", how="right").fillna(0)


df_dia_visita.head()

,person,0,1,2,3,4,5,6
0,0008ed71,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,00091926,2.0,4.0,3.0,6.0,3.0,5.0,11.0
2,00091a7a,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,4.0,0.0,2.0,0.0
4,000c79fe,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [999]:
# Veo cuantas personas a predecir pierdo
df_dia_visita_predecir = pd.merge(df_dia_visita, df_persons, on="person", how="inner")
df_dia_visita_predecir["person"].count() - df_persons["person"].count()

0

In [1000]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 26 columns):
timestamp                   datetime64[ns]
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
month                       int64
day                         int64
day_o

In [1001]:
# Merge

labels_con_dia_semana = pd.merge(df_dia_visita, labels_con_navegador, on="person", how="inner")
labels_con_dia_semana_num = labels_con_dia_semana.drop(columns="person")

### Prueba

In [1016]:
# Separamos la variable a predecir

X, y = labels_con_dia_semana_num.iloc[:,:-1], labels_con_dia_semana_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1017]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [1018]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1019]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1020]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8195278670205336
Test acuracy:  0.8240223463687151
ROC auc score:  0.6630981666514816
Confusion matrix: 
[[1447  286]
 [  29   28]]


In [1021]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8195278670205336
Test acuracy:  0.8240223463687151

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.83      0.90      1733
           1       0.09      0.49      0.15        57

   micro avg       0.82      0.82      0.82      1790
   macro avg       0.53      0.66      0.53      1790
weighted avg       0.95      0.82      0.88      1790


Precision Score:  0.08917197452229299
Recall Score:  0.49122807017543857
F1 Score:  0.1509433962264151
Cohen Kappa Score:  0.10256981604153481
ROC auc score:  0.6630981666514816
Confusion matrix: 
[[1447  286]
 [  29   28]]


### Prueba duplicando los compradores

In [1233]:
# Separamos la variable a predecir

X, y = labels_con_dia_semana_num.iloc[:,:-1], labels_con_dia_semana_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1234]:
# Duplico los compradores

train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1235]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1236]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1237]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1238]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.6818059299191375
Test acuracy:  0.6865921787709497
ROC auc score:  0.7108907583442159
Confusion matrix: 
[[1187  546]
 [  15   42]]


In [1239]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6818059299191375
Test acuracy:  0.6865921787709497

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.68      0.81      1733
           1       0.07      0.74      0.13        57

   micro avg       0.69      0.69      0.69      1790
   macro avg       0.53      0.71      0.47      1790
weighted avg       0.96      0.69      0.79      1790


Precision Score:  0.07142857142857142
Recall Score:  0.7368421052631579
F1 Score:  0.13023255813953488
Cohen Kappa Score:  0.07662218004667487
ROC auc score:  0.7108907583442159
Confusion matrix: 
[[1187  546]
 [  15   42]]


### Lo mejor que llegué ahora si: 71

---

# Time features de Santi

## Hora

#### Cantidad de eventos por hora. (Todas las horas)

In [1182]:
df["day"] = df["timestamp"].dt.day
df["day_of_week"] = df['timestamp'].dt.weekday_name
df['day_of_year'] = df['timestamp'].dt.dayofyear
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
#df_events['second'] = df_events['timestamp'].dt.second
df['week_of_year'] = df['timestamp'].dt.weekofyear

In [1183]:
df_hour = df[['person','hour']]

In [1184]:
df_hour1 = pd.concat([df_hour['person'],pd.get_dummies(df_hour['hour'])],axis = 1).groupby('person').sum()\
            .reset_index()
df_hour1.head()

,person,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00091926,117.0,122.0,33.0,15.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,50.0,10.0,0.0,3.0,7.0,65.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,59.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1185]:
# Merge

labels_con_horas = pd.merge(df_hour1, labels_con_dia_semana, on="person", how="inner")
labels_con_horas_num = labels_con_horas.drop(columns="person")

### Prueba

In [1186]:
# Separamos la variable a predecir

X, y = labels_con_horas_num.iloc[:,:-1], labels_con_horas_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1187]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [1188]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1189]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1190]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8449504120687247
Test acuracy:  0.8558659217877095
ROC auc score:  0.6371265729239428
Confusion matrix: 
[[1509  224]
 [  34   23]]


In [1191]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8449504120687247
Test acuracy:  0.8558659217877095

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.87      0.92      1733
           1       0.09      0.40      0.15        57

   micro avg       0.86      0.86      0.86      1790
   macro avg       0.54      0.64      0.54      1790
weighted avg       0.95      0.86      0.90      1790


Precision Score:  0.0931174089068826
Recall Score:  0.40350877192982454
F1 Score:  0.1513157894736842
Cohen Kappa Score:  0.10500346897880242
ROC auc score:  0.6371265729239428
Confusion matrix: 
[[1509  224]
 [  34   23]]


### Prueba duplicando los compradores

In [1326]:
# Separamos la variable a predecir

X, y = labels_con_horas_num.iloc[:,:-1], labels_con_horas_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1327]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1328]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1329]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1330]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1331]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.6777628032345013
Test acuracy:  0.6815642458100558
ROC auc score:  0.69981069233962
Confusion matrix: 
[[1179  554]
 [  16   41]]


In [1246]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6777628032345013
Test acuracy:  0.6815642458100558

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.68      0.81      1733
           1       0.07      0.72      0.13        57

   micro avg       0.68      0.68      0.68      1790
   macro avg       0.53      0.70      0.47      1790
weighted avg       0.96      0.68      0.78      1790


Precision Score:  0.06890756302521009
Recall Score:  0.7192982456140351
F1 Score:  0.1257668711656442
Cohen Kappa Score:  0.07182169661132576
ROC auc score:  0.69981069233962
Confusion matrix: 
[[1179  554]
 [  16   41]]


#### Cantidad de eventos en fin de semana.

In [1201]:
df_day_of_week3 = df[['person','day_of_week']]

In [1202]:
df_day_of_week3['fin_de_semana'] = df_day_of_week3['day_of_week'].isin(['Saturday','Sunday'])

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1203]:
df_day_of_week3 = df_day_of_week3.groupby('person').sum().reset_index()

In [1247]:
# Merge

labels_con_findes = pd.merge(df_day_of_week3, labels_con_horas, on="person", how="inner")
labels_con_findes_num = labels_con_findes.drop(columns="person")

### Prueba

In [1321]:
# Separamos la variable a predecir

X, y = labels_con_findes_num.iloc[:,:-1], labels_con_findes_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1322]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [1323]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1324]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1325]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8876938119849141
Test acuracy:  0.9005586592178771

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.92      0.95      1733
           1       0.12      0.35      0.18        57

   micro avg       0.90      0.90      0.90      1790
   macro avg       0.55      0.63      0.57      1790
weighted avg       0.95      0.90      0.92      1790


Precision Score:  0.12422360248447205
Recall Score:  0.3508771929824561
F1 Score:  0.18348623853211007
Cohen Kappa Score:  0.14318598634992175
ROC auc score:  0.6347576963181178
Confusion matrix: 
[[1592  141]
 [  37   20]]


### Prueba duplicando los compradores

In [1315]:
# Separamos la variable a predecir

X, y = labels_con_findes_num.iloc[:,:-1], labels_con_findes_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1316]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1317]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1318]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1319]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1320]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6842318059299192
Test acuracy:  0.6916201117318436

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.69      0.81      1733
           1       0.07      0.74      0.13        57

   micro avg       0.69      0.69      0.69      1790
   macro avg       0.53      0.71      0.47      1790
weighted avg       0.96      0.69      0.79      1790


Precision Score:  0.07253886010362694
Recall Score:  0.7368421052631579
F1 Score:  0.13207547169811318
Cohen Kappa Score:  0.07865658865720404
ROC auc score:  0.713487411546755
Confusion matrix: 
[[1196  537]
 [  15   42]]


#### Cantidad de eventos en rango de horas (Morning , Afternoon, Evening y Night) 

Morning     5 a 12  

Afternoon     12 a 17

Evening     17 a 21

Night         21 a 4 

In [1259]:
df_hour_rango2 = df_hour.copy()

In [1260]:
df_hour_rango2['Morning'] = (df_hour_rango2['hour'].isin(list(range(5,13)))).astype(int)
df_hour_rango2['Afternoon'] = (df_hour_rango2['hour'].isin(list(range(13,18)))).astype(int)
df_hour_rango2['Evening'] = (df_hour_rango2['hour'].isin(list(range(18,21)))).astype(int)
df_hour_rango2['Night'] = (df_hour_rango2['hour'].isin([22,23,0,1,2,3,4])).astype(int)
df_hour_rango2.drop(columns = ['hour'],inplace = True)

In [1261]:
df_hour_rango2a = df_hour_rango2.groupby('person').sum().reset_index()

#### Visita la pagina en rango de horas (Morning , Afternoon, Evening y Night) 

In [1262]:
df_hour_rango2b = df_hour_rango2.drop_duplicates().groupby('person').sum().reset_index()

In [1264]:
# Merge

labels_con_rango_horario = pd.merge(df_hour_rango2b, labels_con_findes, on="person", how="inner")
labels_con_rango_horario_num = labels_con_rango_horario.drop(columns="person")

### Prueba

In [1265]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1266]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [1267]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1268]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1269]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.7929878474647297
Test acuracy:  0.7938547486033519

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1733
           1       0.08      0.51      0.14        57

   micro avg       0.79      0.79      0.79      1790
   macro avg       0.53      0.66      0.51      1790
weighted avg       0.95      0.79      0.86      1790


Precision Score:  0.07837837837837838
Recall Score:  0.5087719298245614
F1 Score:  0.1358313817330211
Cohen Kappa Score:  0.08535622793048525
ROC auc score:  0.656001660238305
Confusion matrix: 
[[1392  341]
 [  28   29]]


### Prueba duplicando los compradores

In [1309]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1310]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1311]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1312]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1313]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1314]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6490566037735849
Test acuracy:  0.6581005586592179

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.65      0.79      1733
         1.0       0.07      0.77      0.13        57

   micro avg       0.66      0.66      0.66      1790
   macro avg       0.53      0.71      0.46      1790
weighted avg       0.96      0.66      0.77      1790


Precision Score:  0.06842923794712286
Recall Score:  0.7719298245614035
F1 Score:  0.12571428571428572
Cohen Kappa Score:  0.07138945730178403
ROC auc score:  0.7131432158006094
Confusion matrix: 
[[1134  599]
 [  13   44]]


## Con los datos transformados

In [1283]:
x = labels_con_rango_horario.loc[:,labels_con_rango_horario.columns != 'person']
maxAbsScaler = preprocessing.MaxAbsScaler()
x_scaled = maxAbsScaler.fit_transform(x)
labels_con_rango_horario = pd.concat([labels_con_rango_horario['person'],pd.DataFrame(x_scaled)],axis = 1)
labels_con_rango_horario_num = labels_con_rango_horario.drop(columns="person")

### Prueba

In [1284]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1285]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [1286]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1287]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1288]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.7929878474647297
Test acuracy:  0.7938547486033519

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.80      0.88      1733
         1.0       0.08      0.51      0.14        57

   micro avg       0.79      0.79      0.79      1790
   macro avg       0.53      0.66      0.51      1790
weighted avg       0.95      0.79      0.86      1790


Precision Score:  0.07837837837837838
Recall Score:  0.5087719298245614
F1 Score:  0.1358313817330211
Cohen Kappa Score:  0.08535622793048525
ROC auc score:  0.656001660238305
Confusion matrix: 
[[1392  341]
 [  28   29]]


### Prueba duplicando los compradores

In [1301]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1304]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1305]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1306]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1307]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1308]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.5872576177285319
Test acuracy:  0.5687150837988827

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.56      0.72      1733
         1.0       0.06      0.84      0.11        57

   micro avg       0.57      0.57      0.57      1790
   macro avg       0.52      0.70      0.41      1790
weighted avg       0.96      0.57      0.70      1790


Precision Score:  0.059186189889025895
Recall Score:  0.8421052631578947
F1 Score:  0.11059907834101382
Cohen Kappa Score:  0.054326864513373874
ROC auc score:  0.7009141434081453
Confusion matrix: 
[[970 763]
 [  9  48]]


# -------- Borrador -----------------

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc"

In [ ]:
params = {"objective":"binary:hinge",'colsample_bytree': 0.8,
          'learning_rate': 0.1, 'max_depth': 5, "n_estimators" : 6}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10,
                    num_boost_round=10,
                    metrics="auc", as_pandas=True, seed=123);

In [ ]:
#cv_results.sort_values("train-auc-mean", ascending=False)
cv_results

In [ ]:
tf_test = pd.read_csv("trocafone_kaggle_test.csv")
tf_test.head()

## Cantidad de entradas desde campañas

In [397]:
df_events_campaign = df[['person','campaign_source']].dropna()


top_5_campanias = df_events_campaign['campaign_source'].value_counts().head(5).index
df_events_campaign = df_events_campaign.loc[df_events_campaign['campaign_source'].isin(top_5_campanias)]
df_events_campaign = pd.concat([df_events_campaign['person'],\
        pd.get_dummies(df_events_campaign['campaign_source'])],axis = 1).groupby('person').sum().reset_index()

df_events_campaign = pd.merge(df_events_campaign, todas_las_personas, on="person", how="right").fillna(0)


df_events_campaign.head()

,person,criteo,emblue,google,rtbhouse,zanox
0,00091926,1.0,0.0,13.0,1.0,0.0
1,00091a7a,0.0,0.0,1.0,0.0,0.0
2,000ba417,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,1.0,0.0,0.0
4,000e4d9e,0.0,0.0,4.0,0.0,9.0


In [398]:
# Veo cuantas personas a predecir pierdo
df_events_campaign_predecir = pd.merge(df_events_campaign, df_persons, on="person", how="inner")
df_events_campaign_predecir["person"].count() - df_persons["person"].count()

0

In [405]:
# Merge
labels_con_camps = pd.merge(df_events_campaign, labels_con_dia_semana, on="person", how="inner")
labels_con_camps_num = labels_con_camps.drop(columns="person")
labels_con_camps_num.head()


,criteo,emblue,google,rtbhouse,zanox,0,1,2,3,4,...,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,4,9,1,17,17,16,17,17,0
1,5.0,0.0,17.0,7.0,0.0,3.0,6.0,2.0,2.0,3.0,...,12,63,189,15,471,290,358,290,72,0
2,3.0,0.0,4.0,0.0,0.0,0.0,3.0,6.0,3.0,0.0,...,6,20,52,1,96,54,64,54,17,0
3,2.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,10,8,17,3,54,5,26,46,22,0
4,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1,3,17,1,38,26,25,26,26,0


### Prueba

In [406]:
# Separamos la variable a predecir

X, y = labels_con_camps_num.iloc[:,:-1], labels_con_camps_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [407]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [408]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [409]:
# Predecimos

preds = xg_reg.predict(X_test)

In [410]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8869953904176561
Test acuracy:  0.8748603351955307
ROC auc score:  0.5785714285714286
Confusion matrix: 
[[1548  172]
 [  52   18]]


## Cantidad de visitas a productos Apple y Samsung

In [365]:
df_productos = df[["person", "event", "model"]]
df_productos = df_productos.loc[df_productos["event"] == "viewed product"]
df_productos = df_productos.loc[(df_productos["model"].str.contains("iPhone") )|( df_productos["model"].str.contains("Samsung") )]
df_productos.head()

df_apple = df_productos.loc[df_productos["model"].str.contains("iPhone")]
df_samsung = df_productos.loc[df_productos["model"].str.contains("Samsung")]                           

vistas_apple = df_apple["person"].value_counts().reset_index()
vistas_apple.columns = ["person", "vistas_apple"]

vistas_samsung = df_apple["person"].value_counts().reset_index()
vistas_samsung.columns = ["person", "vistas_samsung"]

vistas_prods = pd.merge(vistas_samsung, vistas_apple, on="person", how="inner")

vistas_prods = pd.merge(vistas_prods, todas_las_personas, on="person", how="right").fillna(0)

vistas_prods.head()

,person,vistas_samsung,vistas_apple
0,6abd2bf1,2051.0,2051.0
1,b1f4dbf6,1866.0,1866.0
2,9bf968c5,1557.0,1557.0
3,bfb74b38,1526.0,1526.0
4,6f19cfd9,1442.0,1442.0


In [366]:
# Veo cuantas personas a predecir pierdo
vistas_prods_predecir = pd.merge(vistas_prods, df_persons, on="person", how="inner")
vistas_prods_predecir["person"].count() - df_persons["person"].count()

0

In [375]:
# Merge

df_con_vistas_prods = pd.merge(vistas_prods,labels_con_dia_semana, on="person", how="inner")

df_con_vistas_prods_num = df_con_vistas_prods.drop(columns="person")

### Prueba

In [376]:
# Separamos la variable a predecir

X, y = df_con_vistas_prods_num.iloc[:,:-1], df_con_vistas_prods_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [377]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6)


In [378]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [379]:
# Predecimos

preds = xg_reg.predict(X_test)

In [380]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9113004609582344
Test acuracy:  0.9016759776536313
ROC auc score:  0.582583419606594
Confusion matrix: 
[[1600  132]
 [  44   14]]


## Cantidad de busquedas en 'searched products' y 'search engine hit'

In [383]:
df_search = df.loc[(df["event"] == "searched products" )| (df["event"] == "search engine hit" )]

searched_products = df_search.loc[df_search["event"] == "searched products"]
cant_searched_products = searched_products["person"].value_counts().reset_index()
cant_searched_products.columns = ["person", "cant_search_prod"]

engine_hit = df_search.loc[df_search["event"] == "search engine hit"]
cant_engine_hit = engine_hit["person"].value_counts().reset_index()
cant_engine_hit.columns = ["person", "cant_engine_hit"]

cant_busquedas = pd.merge(cant_engine_hit, cant_searched_products, on="person", how="inner")

cant_busquedas = pd.merge(cant_busquedas, todas_las_personas, on="person", how="right").fillna(0)



In [384]:
# Veo cuantas personas a predecir pierdo
cant_busquedas_predecir = pd.merge(cant_busquedas, df_persons, on="person", how="inner")
cant_busquedas_predecir["person"].count() - df_persons["person"].count()

0

In [391]:
# Merge

labels_con_busq = pd.merge(cant_busquedas, labels_con_dia_semana, on="person", how="inner")
labels_con_busq_num = labels_con_busq.drop(columns="person")
labels_con_busq_num.head()

,cant_engine_hit,cant_search_prod,0,1,2,3,4,5,6,Chrome 65.0,...,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,245.0,3.0,22.0,15.0,22.0,16.0,28.0,19.0,19.0,4.0,...,245,170,744,47,2006,388,378,861,198,0
1,136.0,739.0,9.0,8.0,14.0,13.0,6.0,10.0,6.0,0.0,...,136,61,739,10,1609,1609,1608,1609,191,0
2,130.0,214.0,18.0,14.0,12.0,14.0,7.0,9.0,12.0,0.0,...,130,87,214,4,855,760,759,760,138,0
3,102.0,2.0,17.0,21.0,24.0,24.0,21.0,14.0,14.0,29.0,...,102,130,450,2,1025,103,105,585,102,0
4,93.0,121.0,26.0,23.0,33.0,20.0,33.0,23.0,7.0,56.0,...,93,416,692,17,1593,412,420,956,216,0


### Prueba

In [392]:
# Separamos la variable a predecir

X, y = labels_con_busq_num.iloc[:,:-1], labels_con_busq_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [393]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 100, min_child_weight=1)


In [394]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1000, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=100, seed=None,
       silent=True, subsample=1)

In [395]:
# Predecimos

preds = xg_reg.predict(X_test)

In [396]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8112864925268892
Test acuracy:  0.8011173184357542
ROC auc score:  0.6384819951317123
Confusion matrix: 
[[1403  320]
 [  36   31]]


### duplico compradores

In [1022]:
# Separamos la variable a predecir

X, y = labels_con_busq_num.iloc[:,:-1], labels_con_busq_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1023]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1024]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1025]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1026]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1027]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7886639676113361
Test acuracy:  0.764804469273743
ROC auc score:  0.6339645359967429
Confusion matrix: 
[[1336  387]
 [  34   33]]


In [1028]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.7886639676113361
Test acuracy:  0.764804469273743

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.78      0.86      1723
           1       0.08      0.49      0.14        67

   micro avg       0.76      0.76      0.76      1790
   macro avg       0.53      0.63      0.50      1790
weighted avg       0.94      0.76      0.84      1790


Precision Score:  0.07857142857142857
Recall Score:  0.4925373134328358
F1 Score:  0.135523613963039
Cohen Kappa Score:  0.07585995462628004
ROC auc score:  0.6339645359967429
Confusion matrix: 
[[1336  387]
 [  34   33]]
